# BERT 학습을 위한 전처리

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 8.0 MB/s 
     |████████████████████████████████| 163 kB 65.8 MB/s 
     |████████████████████████████████| 7.6 MB 47.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 7.6 MB/s 
     |████████████████████████████████| 115 kB 36.9 MB/s 
     |████████████████████████████████| 212 kB 67.1 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentence1 = "What a beautiful day!"
sentence2 = "Nvidia Titan XP has 12GB of VRAM"

# 1. 토큰화 결과
print(sentence1, '토큰화 결과:', tokenizer.tokenize(sentence1))
print(sentence2, '토큰화 결과:', tokenizer.tokenize(sentence2))

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

What a beautiful day! 토큰화 결과: ['what', 'a', 'beautiful', 'day', '!']
Nvidia Titan XP has 12GB of VRAM 토큰화 결과: ['n', '##vid', '##ia', 'titan', 'xp', 'has', '12', '##gb', 'of', 'vr', '##am']


In [ ]:
# 2. BERT 모형 입력 생성
inputs = tokenizer([sentence1, sentence2], padding=True)
print('BERT 입력:', inputs)

BERT 입력: {'input_ids': [[101, 2054, 1037, 3376, 2154, 999, 102, 0, 0, 0, 0, 0, 0], [101, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
# 3. 두 문장으로 이루어진 시퀀스에 대한 BERT 모형 입력 생성
inputs = tokenizer(sentence1, sentence2, padding=True)
print('두 문장 시퀀스에 대한 BERT 입력:', inputs)

두 문장 시퀀스에 대한 BERT 입력: {'input_ids': [101, 2054, 1037, 3376, 2154, 999, 102, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


# 트랜스포머의 트레이너를 이용한 미세조정학습

**NLTK 영화리뷰 데이터 준비**

In [ ]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split    # sklearn에서 제공하는 split 함수를 사용
import numpy as np

nltk.download('movie_reviews')
fileids = movie_reviews.fileids()   # movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids]     # file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = [label_dict[c] for c in categories]

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


Train set count:  1600
Test set count:  400


**토크나이저 및 모델(분류기) 설정**

In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification 

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")    # BERT모델에서 정한 입력 크기보다 크면 잘라내기, 패딩 채우기
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

**데이터셋 변환**

In [ ]:
import torch

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = OurDataset(train_input, y_train)
test_dataset = OurDataset(test_input, y_test)

**정확도 함수**

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")    # Trainer는 손실값만 제공함

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


**미세조정 학습**

In [ ]:
from transformers import Trainer, TrainingArguments

# Trainer에서 사용할 하이퍼 파라미터 지정
training_args = TrainingArguments(
    output_dir='./results',          # 모형 예측이나 체크포인트 출력 폴더, 반드시 필요함
    num_train_epochs=5,              # 학습 에포크 수
    per_device_train_batch_size=8,   # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=8,   # 평가에 사용할 배치 사이즈
)

trainer = Trainer(
    model=model,                     # 학습할 모형
    args=training_args,              # 위에서 정의한 학습 매개변수
    train_dataset=train_dataset,     # 훈련 데이터셋
    compute_metrics=compute_metrics,
)

# 미세조정 학습 실행
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1600
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Step,Training Loss
500,0.358400
1000,0.045500


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1000, training_loss=0.20195878410339355, metrics={'train_runtime': 863.1752, 'train_samples_per_second': 9.268, 'train_steps_per_second': 1.159, 'total_flos': 2104888442880000.0, 'train_loss': 0.20195878410339355, 'epoch': 5.0})

**모델 평가**

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 400
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


{'eval_loss': 0.6868287920951843,
 'eval_accuracy': 0.905,
 'eval_runtime': 14.922,
 'eval_samples_per_second': 26.806,
 'eval_steps_per_second': 3.351,
 'epoch': 5.0}

# 파이토치를 이용한 미세조정학습

In [ ]:
# GPU 메모리 확보 (트레이너를 이용한 미세조정학습을 실행했다면 필수)

del model
del trainer
torch.cuda.empty_cache()

**데이터로더 구축**

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8)

**사전 학습 모델 가져오기**

In [ ]:
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.config

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/5546055f03398095e385d7dc625e636cc8910bf2/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/5546055f03398095e385d7dc625e636cc8910bf2/pytorch_model.bin
Some weights of the model check

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

**모델 정의**

In [ ]:
# BERT를 포함한 신경망 모형

class MyModel(torch.nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels): 
        super(MyModel, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        self.pretrained_model = pretrained_model

        # 분류기 정의
        self.classifier = torch.nn.Linear(self.token_size, self.num_labels)

    def forward(self, inputs):
        # BERT 모형에 입력을 넣고 출력을 받음
        outputs = self.pretrained_model(**inputs)
        # BERT 출력에서 CLS 토큰에 해당하는 부분만 가져옴
        bert_clf_token = outputs.last_hidden_state[:,0,:]
        
        # BERT 모형에 분휴기 추가해서 출력
        return self.classifier(bert_clf_token)

# token_size는 BERT 토큰과 동일
model = MyModel(bert_model, num_labels=2, token_size=bert_model.config.hidden_size) 

**모델 학습**

In [ ]:
from transformers import AdamW
import torch.nn.functional as F
import time

# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)  # 모형을 GPU로 복사
model.train()     # 학습모드로 전환

optim = AdamW(model.parameters(), lr=5e-5)  # 옵티마이저를 트랜스포머가 제공하는 AdamW로 설정
criterion = torch.nn.CrossEntropyLoss()     # 멀티클래스이므로 크로스 엔트로피를 손실함수로 사용

start = time.time() # 시작시간 기록
num_epochs = 5      # 학습 epoch를 3회로 설정

for epoch in range(num_epochs):
    total_epoch_loss = 0  # epoch의 총 loss 초기화

    for step, batch in enumerate(train_loader):
        optim.zero_grad()     # 그래디언트 초기화

        # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = model(inputs)     # 모형으로 결과 예측

        # 두 클래스에 대해 예측하고 각각 비교해야 하므로 labels에 대해 원핫인코딩을 적용한 후에 손실을 게산
        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float()) # loss 계산

        if (step+1) % 100 == 0:     # 100 배치마다 경과한 시간과 loss를 출력
            elapsed = time.time() - start
            print('Epoch %d, batch %d, elapsed time: %.2f, loss: %.4f' % (epoch+1, step+1, elapsed, loss))

        total_epoch_loss += loss
        loss.backward()     # 그래디언트 계산
        optim.step()        # 가중치 업데이트

    avg_epoch_loss = total_epoch_loss / len(train_loader)   # epoch의 평균 loss 계산

    print('Average loss for epoch %d: %.4f' % (epoch+1, avg_epoch_loss))

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Epoch 1, batch 100, elapsed time: 83.95, loss: 0.7491
Epoch 1, batch 200, elapsed time: 168.37, loss: 0.6772
Average loss for epoch 1: 0.6416
Epoch 2, batch 100, elapsed time: 252.53, loss: 0.5133
Epoch 2, batch 200, elapsed time: 337.10, loss: 0.3826
Average loss for epoch 2: 0.3994
Epoch 3, batch 100, elapsed time: 421.69, loss: 0.3940
Epoch 3, batch 200, elapsed time: 506.29, loss: 0.0680
Average loss for epoch 3: 0.2013
Epoch 4, batch 100, elapsed time: 590.92, loss: 0.0149
Epoch 4, batch 200, elapsed time: 675.58, loss: 0.0061
Average loss for epoch 4: 0.1118
Epoch 5, batch 100, elapsed time: 760.37, loss: 0.0020
Epoch 5, batch 200, elapsed time: 844.91, loss: 0.0103
Average loss for epoch 5: 0.0699


**모델 평가**

In [ ]:
from datasets import load_metric

# 테스트 데이터 데이터로더 변환
test_loader = DataLoader(test_dataset, batch_size=8)

metric = load_metric("accuracy")
model.eval()

for batch in test_loader:
    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    
    with torch.no_grad():   # 학습 X (그래디언트 계산 X)
        outputs = model(inputs)

    predictions = torch.argmax(outputs, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

metric.compute()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


{'accuracy': 0.845}